In [1]:
from copyreg import pickle
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
import requests
import io
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
imputer_num = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
imputer_cat = SimpleImputer(strategy="most_frequent")
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from flytekit import task, workflow
from joblib import dump
from sklearn.preprocessing import OneHotEncoder
from typing import Tuple
import pickle
hi=None

In [2]:
url="https://github.com/smadarab/flytelab/raw/main/census 2.csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')),sep=',')
print("df is created",df.columns)
#df.dropna(inplace=True)
#df = df.reset_index()
train=df
print(df.columns)

df is created Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


In [5]:
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loss', 'hours-per-week']
cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
log_transform_cols = ['capital-loss', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  
def one_hot_encode(X):
    print("one hot encode")
    ohe = OneHotEncoder(handle_unknown = 'ignore')
    ohe.fit(pd.DataFrame(X))
    
    return ohe.transform(pd.DataFrame(X)).toarray()

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#    ('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])


In [6]:
full_pipeline = Pipeline([('steps_', steps_)])
y = train['income'].map({'<=50K': 0, '>50K': 1})
X = full_pipeline.fit_transform(train)
model = AdaBoostClassifier(n_estimators=300)
X_train, X_test, y_train, y_test = train_test_split(X, y)
model.fit(X_train, y_train)

one hot encode


AdaBoostClassifier(n_estimators=300)

In [8]:
X_train[0]

array([0.        , 0.        , 0.28767123, 0.66666667, 0.        ,
       0.        , 0.44897959, 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [9]:
y_pred=model.predict_proba(X_test)

In [10]:
y_pred = [list(i).index(max(list(i))) for i in y_pred]

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [12]:
pd.Series(y_pred).value_counts()

0    9012
1    2294
dtype: int64

In [13]:
precision_score(y_pred,y_test)

0.6471663619744058

In [14]:
recall_score(y_pred,y_test)

0.7715780296425457

In [15]:
df.columns

Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [16]:
df[df['income']>'50k'].iloc[0]

age                            39
workclass               State-gov
education_level         Bachelors
education-num                13.0
marital-status      Never-married
occupation           Adm-clerical
relationship        Not-in-family
race                        White
sex                          Male
capital-gain               2174.0
capital-loss                  0.0
hours-per-week               40.0
native-country      United-States
income                      <=50K
Name: 0, dtype: object

In [17]:
age=39
education_num=13.0
capital_gain=2174.0
capital_loos=0.0
hour_per_week=40.0
workclass='State-gov'
marital_status='Never-married'
occupation='Adm-clerical'
relationship='Not-in-family'
race='White'
sex='Male'
native_country = 'United-States'
education_level='Bachelors'

In [18]:
X_train = pd.DataFrame({'age': age, 'education-num': education_num,'education_level':education_level,'capital-gain':capital_gain,'capital-loss':capital_loos,'hours-per-week':hour_per_week,'workclass':workclass,'marital-status':marital_status,'occupation':occupation,'relationship':relationship,'race':race,'sex':sex,'native-country':native_country},index=[0])


In [19]:
X_train

,age,education-num,education_level,capital-gain,capital-loss,hours-per-week,workclass,marital-status,occupation,relationship,race,sex,native-country
0,39,13.0,Bachelors,2174.0,0.0,40.0,State-gov,Never-married,Adm-clerical,Not-in-family,White,Male,United-States


In [20]:
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loss', 'hours-per-week']
cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
log_transform_cols = ['capital-loss', 'capital-gain']      
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    return X[log_transform_cols]

def one_hot_encode(X):
    return ohe.transform(pd.DataFrame(X)).toarray()

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
#('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
#('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
#('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])

In [21]:
full_pipeline = Pipeline([('steps_', steps_)])
X_train = full_pipeline.fit_transform(X_train)

NameError: name 'ohe' is not defined

In [63]:
X_train

array([[0.        , 7.68478394, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [64]:
y_pred=model.predict_proba(X_train)

In [65]:
final = y_pred.tolist()[0].index(max(y_pred.tolist()[0]))

In [66]:
final

0

In [72]:
pd.DataFrame()

/Users/apple/flytelab/projects/my_project/env/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


,0
0,


In [81]:
hi = pd.DataFrame.sparse.from_spmatrix(hi.transform(X_train[cat_cols]))

/Users/apple/flytelab/projects/my_project/env/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [87]:
hi.values

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])